In [22]:
#!pip3 install scikit-surprise

In [8]:
import numpy as np
import pandas as pd
"""from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate
import copy
from datetime import datetime"""
import pickle
import warnings
warnings.filterwarnings("ignore")

In [9]:
meta=pd.read_csv('meta.csv')

# User Based Recommander: Recommand Now

In [10]:
with open('df_rec .pickle', 'rb') as handle:
    df_rec = pickle.load(handle)
print(df_rec)

       userId  movieId  score_norm_4     original_title
0           1     1371      1.967574          Rocky III
1           4     1371      3.906171          Rocky III
2           7     1371      3.003806          Rocky III
3          19     1371      3.106015          Rocky III
4          21     1371      3.062812          Rocky III
...       ...      ...           ...                ...
44999     652   129009      4.000000     Love Is a Ball
45000     653     2103      3.000000            Solaris
45001     659      167      4.000000              K-PAX
45002     659      563      3.000000  Starship Troopers
45003     665      129      3.000000           千と千尋の神隠し

[45004 rows x 4 columns]


In [11]:
def collab_recommander(user,N):
    return df_rec[df_rec['userId'] == user].sort_values(by=['score_norm_4'], ascending=False).head(N)

In [12]:
collab_recommander(652,10).loc[:,["original_title"]]

,original_title
44986,The Return of Doctor X
44990,Lion of the Desert
44988,Anchorman: The Legend of Ron Burgundy
44998,8:46
44989,Brigham City
44996,Mud
43021,Carts of Darkness
35284,Irma la Douce
33204,Le fabuleux destin d'Amélie Poulain
44270,Tanguy


# Content Based Recommander: Recommand Now

In [13]:
import pickle 
import pandas as pd

In [14]:
with open('matrix_vocab.pickle', 'rb') as handle:
    matrix_vocab = pickle.load(handle)
#matrix_vocab
df = pd.read_csv('content.csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.head(5)


,adult,genres,imdb_id,original_title,overview
0,['false'],"['family', 'comedy', 'animation']",tt0114709,Toy Story,"['owner', 'learns', 'live', 'separate', 'room'..."
1,['false'],"['family', 'adventure', 'fantasy']",tt0113497,Jumanji,"['unwittingly', 'three', 'magical', 'room', 'h..."
2,['false'],"['comedy', 'romance']",tt0113228,Grumpier Old Men,"['max', 'fish', 'reignites', 'less', 'scare', ..."
3,['false'],"['comedy', 'romance', 'drama']",tt0114885,Waiting to Exhale,"['good', 'waiting', 'man', 'elusive', 'determi..."
4,['false'],['comedy'],tt0113041,Father of the Bride Part II,"['wife', 'plan', 'news', 'daughter', 'banks', ..."


In [15]:
import gensim
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [16]:
def content_recommander(movie_title,data,N): 
    matrix_vocab = []
    for list_ in data.to_numpy():
        list_[0] = [word for word in list_[0] if word in wv.key_to_index]
        list_[1] = [word for word in list_[1] if word in wv.key_to_index]
        list_[4] = [word for word in list_[4] if word in wv.key_to_index]
        matrix_vocab.append(list_)
    data_vocab = pd.DataFrame(matrix_vocab, columns=data.columns)
    
    
    matrix_title_vocab = []
    for list_ in data[data['original_title'] == movie_title].to_numpy():
        list_[0] = [word for word in list_[0] if word in wv.key_to_index]
        list_[1] = [word for word in list_[1] if word in wv.key_to_index]
        list_[4] = [word for word in list_[4] if word in wv.key_to_index]
        matrix_title_vocab.append(list_)

    matrix_similarity = []
    pbar = matrix_vocab
    for list1 in pbar:
        for list2 in matrix_title_vocab:  
            try:
                score_adult = wv.n_similarity(list1[0], list2[0])
            except:
                score_adult = 0
            try:
                score_genre = wv.n_similarity(list1[1], list2[1])
            except:
                score_genre = 0
            try:
                score_overview = wv.n_similarity(list1[4], list2[4])
            except:
                score_overview = 0
            if ((list1[4] != list2[4]) ):
                matrix_similarity.append([ list1[3], list2[3], list1[0], list2[0],list1[1], list2[1],list1[4], list2[4], score_adult, score_genre,score_overview])

    dfsimilarity = pd.DataFrame(matrix_similarity, columns = ['original_title','movie_request','adult','adult_request','genre','genre_request','overview','overview_request','score_adult', 'score_genre','score_overview'])
    dfsimilarity['final_score'] = dfsimilarity['score_adult'] + dfsimilarity['score_genre']+ dfsimilarity['score_overview']
    return (dfsimilarity.sort_values(by=['final_score',  'score_genre', 'score_overview'], ascending=False).head(N))

In [17]:
rec = content_recommander('Richard III',df,5)

In [18]:
rec["original_title"]

107618        Forty Little Mothers
10546             Sans toit ni loi
64616               Doctors' Wives
115862    Ubistvo s predumišljajem
9169              Any Given Sunday
Name: original_title, dtype: object

# Hybrid Recommander In Cascade: Recommand Now

In [19]:
def hybrid_recommander(user,df,N):
    x = collab_recommander(user,2)
    y = x[x['score_norm_4']>3.5]
    #print(y)
    items_favoris = list(set(y['original_title'].tolist()))
    #print(items_favoris)
    items_sim = []
    for i in tqdm(range(len(items_favoris))):
        items_sim.append(content_recommander(items_favoris[i],df,5))#.head(5))
    content= pd.concat(items_sim[0:len(items_favoris)]).sort_values(by=['final_score'], ascending = False)
    #print(content)
    fin = content.sort_values(by='final_score',ascending = False)    
    return fin['original_title'].head(N)

In [20]:
from tqdm import tqdm

In [21]:
hybrid_recommander(652, df,5)

100%|██████████| 2/2 [02:05<00:00, 61.30s/it]


1869     Friday the 13th: A New Beginning
29865                            The Dorm
13720                     Drag Me to Hell
3831                           Lost Souls
13526                          Naked Fear
Name: original_title, dtype: object